In [1]:
import json
import re
# from nltk.tokenize import sent_tokenize, word_tokenize


In [2]:
def load_processed_tweets_from_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

data = load_processed_tweets_from_json("gg2013_preprocessed.json")

In [3]:

import nltk
from nltk.corpus import words
nltk.download('words')

english_words = set(words.words())

def is_dictionary_word(string):
    
    if string == "-":
        return True
    return string.lower() in english_words


def extract_award_name(sentence):
    """Extracts the part of the sentence based on specified conditions."""
    # Find all hyphen positions
    hyphen_positions = [i for i, char in enumerate(sentence) if char == '-']
    res = None

    for index, pos in enumerate(hyphen_positions):
        # Split the sentence at the current hyphen position
        left_part = sentence[:pos].strip()
        right_part = sentence[pos + 1:].strip()

        # Check if there's something on the right
        if not right_part:
            return res

        # Split left part into words
        left_words = left_part.split()

        # Check conditions
        if (any(word.lower() in left_words for word in ['best', 'award']) and
            all(is_dictionary_word(word) for word in left_words if word)):  # Ensure no empty strings are checked
            
            if index < len(hyphen_positions) - 1:
                rr = sentence[pos+1:hyphen_positions[index + 1]]
            else:
                rr = sentence[pos+1:]

            res = [left_part.strip(), rr ]
        else:
            return res

    return res


print(extract_award_name("best supporting actor in a motion picture - christoph waltz - django unchained - goldenglobes"))
print(extract_award_name("best supporting actor in a motion picture - comedy or picture - christoph waltz - django unchained - goldenglobes"))
print(extract_award_name("best supporting actor in a motion picture - christoph waltz"))







['best supporting actor in a motion picture', ' christoph waltz ']
['best supporting actor in a motion picture - comedy or picture', ' christoph waltz ']
['best supporting actor in a motion picture', ' christoph waltz']


[nltk_data] Error loading words: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [4]:
def extract_winner_info(sentence):
    """Check if the sentence is in the format '... wins ...' with 'best' or 'award' in the second part."""

    pattern = r'^(.*?)(wins|receives)(.*?)(best|award)(.*?)$'
    match = re.search(pattern, sentence, re.IGNORECASE)

    if match:

        first_part = match.group(1).strip()
        second_part = match.group(3).strip() + " " + match.group(
            4).strip()  + match.group(5).strip()
        return [first_part, second_part]

    return []

def get_combinations(input_string):
   
    words = input_string.split()
    combinations = []
    
    

    for j in range(1, len(words) + 1):
        
        combinations.append(' '.join(words[0:j]))
    
    return combinations

In [5]:
for index, tweet in enumerate(data):
    
    tweet_text = tweet['text']
    
    award_name = extract_winner_info(tweet_text)
    
    if award_name:
        win_resolutions = {
            "award" : get_combinations(award_name[1]),
            "winner": [award_name[0]],
            "confidence": 0.7
            
        }
        tweet["win_resolutions"] = win_resolutions
        
        continue
    award_name = extract_award_name(tweet_text)
    
    if award_name:
        win_resolutions = {
            "award" : get_combinations(award_name[0]),
            "winner": [award_name[1]],
            "confidence": 0.5
            
        }
        tweet["win_resolutions"] = win_resolutions

        
        

In [6]:
%pip install blis

Note: you may need to restart the kernel to use updated packages.


In [7]:
!python -m spacy download en_core_web_sm


/System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: No module named spacy


In [8]:
import spacy
nlp = spacy.load("en_core_web_sm")
def get_part_before_host(sentence):
    # pattern to capture only the part before the hosting-related verbs
    pattern = r"(.*?)(?:\s+(?:is|are)\s+)?(?:hosts|host|are hosting|is hosting)"

    
    match = re.search(pattern, sentence, re.IGNORECASE)

    if match:
        # Return the part before the hosting verb
        return match.group(1).strip()

    return None


def find_hosts_from_text(text):

    hosts = []
    # Skip if no mention of host in text, or about next year
    if re.search(r'(?:\bhost)|(?:host(?:s|ed|ing)?\b)', text, re.IGNORECASE) == None \
        or re.search(r'\bnext\b', text, re.IGNORECASE) != None:
        return hosts
    
    doc = nlp(text)

    for sentence in doc.sents:

        # Find the root verb of the sentence
        root_verb = [token for token in sentence if token.dep_ == "ROOT"]

        if root_verb and root_verb[0].lemma_ == "host":
            # Find the people mentioned in the sentence
            potential_hosts = [entity for entity in doc.ents \
                               if entity.label_ == "PERSON"]
            # Add to hosts
            hosts.extend([host.text for host in potential_hosts])
            break

            # # Get the part before the hosting verb using regex function
            # sentence_text = str(sentence)
            # before_host = get_part_before_host(sentence_text)

            # if before_host:
            #     # Split the part by "and" or "&" to extract multiple hosts
                
            #     potential_hosts = re.split(r'\s*(?:and|&|/|\\)\s*', before_host)

            #     hosts.extend(potential_hosts)

    return hosts



In [10]:
for index, tweet in enumerate(data):
    
    tweet_text = tweet['text']
    
    
    hosts = find_hosts_from_text(tweet_text)
    
    if hosts:
        host_resolutions = {
            "hosts": hosts,
            "confidence": 0.7
            
            
        }
        tweet["host_resolutions"] = host_resolutions
        
    
        
        

In [11]:
filename = 'parsed_data.json'

# Write data to JSON file
with open(filename, 'w') as json_file:
    json.dump(data, json_file, indent=4)